# Supervised WSD. Naïve Bayes Classification.
*Author: Daniel Nova*

## Background
The Naïve-Bayes Classifier is a supervised machine learning model based on the Bayes Theorem with a strong independence assumption between the features (hence the "naïve" name). Based on the $n$ features of an observation $x$,  $\vec{x}=(x_1,x_2,...,x_n)$, this classifier estimates to which class $c$ observation $x$ belongs to. We can compute the conditional probability of $x$ belonging to a class $c$ as:
$$P(c|x)=\frac{P(x|c)P(c)}{P(x)}$$

* $P(x|c)$ being the **likelihood** of $x$ belonging to $c$.
* $P(c)$ being the **prior** probability of class $c$.
* $P(x)$ being the **evidence**.

The goal of the classifier is to find the **best class**, maximum a-posteriori class (MAP),  given our observation.
$$c_{MAP}=argmax_{c\in C} P(c|x)$$

Given that we use the Bayesian interpretation to compute this probability, and the denominator can be ignored because its value is the same for all possible $x$'s, our task becomes:
$$c_{MAP}=argmax_{c\in C} P(x_1,x_2,...,x_n|c)P(c)$$

Based on the naïve feature independence assumption, the likelihood $P(x|c)$ can be formulated as:
$$P(x|c) = \prod_{i=1}^{n}P(x_i|c)$$
With this new  interpretation, the  Naïve-Bayes Classifier can now be expressed as:
$$c_{nb}=argmax_{c_j\in C}  P(c_j) \prod_{x \in X} P(x|c)$$

## Supervised Word Sense Disambiguation
From a Machine Learning perspective WSD involves a **classification** task, we want to classify a word as belonging to a class that represents its sense. When we have a labeled corpus that contains the correct word senses, we can perform WSD using a supervised ML approach, to do it we extract useful features, train, and test a classifier.

Based on the Naive Bayes formulation we did before, we interpret this classification problem as:
$$\hat{s}_{nb}=argmax_{s\in S} P(s) \prod_{i=1}^{n} P(x_i|s)$$
Where:
* $\hat{s}$ is the estimated sense
* $P(s_j)$ can be computed by counting the number of occurrences of $s_j$ divided by the count of the target word $w$ :
	$$P(s_j) = \frac{count(s_j, w)}{count(w)}$$
* $P(x_i|s)$ the likelihood of the set of features $x_i$ belonging to sense $s$. Which can be computed as: 
	$$P(x_i|s) = \frac{count(x_i|, s)}{count(s)} $$

## Feature Selection
As in any supervised machine learning task, it is crucial to select  useful features to predict word senses. For determining the sense we have to look at its context, in this case the surrounding text around a target word; from this context we can extract two kinds of features: *collocational* and *bag-of-words*.
* *Collocational features* are taken from the words surrounding the target word, these are chosen based on a **window** to determine how many words to consider. To build the feature vector we can include the word, its root form, and its part of speech (POS). Using a window of two this vector takes the form:
$$[w_{i-2}, POS_{i-2}, w_{i-1}, POS_{i-1}, w_{i+1}, POS_{i+1}, w_{i+2}, POS_{i+2}]$$

e.g. of the surrounding words for text *guitar and bass player stand* for word *bass*
``['guitar', 'NN', 'and', 'CC', 'player', 'NN', 'stand', 'VB']``

* *Bag-of-words* we take into consideration the surrounding text for a given target word but do not care about their position, instead from a set context region we create an ordered word vector, excluding stopwords, and build the features. We want the counts of the words, to do it we use a **co-occurences** vector.

We need to create a vector with the terms that co-occur the most for a given class in our corpus, which works as a *reference* vector. To build the features of an observation we use the most frequent words for that instance and build the vector based on the *reference* we created before, in other words if the most frequent terms in the instance occur in the *reference* vector we include this term in the co-occurrences vector.

To give an example of a co-occurence vector, let's take a look at this example from Jurafsky and Martin's book:
We have a 12 words vocuabluary of the most frequent words for **bass**:
```
[fishing, big, sound, player, fly, rod, pound, double, runs, playing, guitar, band] 
```
So the co-occurence vector for a sentence *"guitar and bass player stand"* would be:
$$[0,0,0,1,0,0,0,0,0,0,1,0]$$

# Taking a look at the data
Now we'll take a look at our Senseval corpus that we downloaded from the NLTK website. If you haven't downloaded it yet follow the instructions at <http://www.nltk.org/data.html>
To begin let's import the needed packages.

In [1]:
import nltk
from nltk.corpus import senseval

files_list = senseval.fileids()
files_list

['hard.pos', 'interest.pos', 'line.pos', 'serve.pos']

This corpus consists of 4 files, each containing senses for 4 different word tokens: **hard, interest, line**, and **serve**.
Each instance consists of:
* Target word in its root form. Marked with a ``<head>`` tag in the corpus file.
* Senses: The ids of the senses of the target word. 
* Context. A sentence that contains the target word.
* Position: Position of target word in the context
* POS (part-of-speech) tags for each token.

NLTK already has a corpus reader for Senseval data, you can learn more about NLTK's Corpus Readers at: <http://www.nltk.org/howto/corpus.html>. Let' check the first instance for *hard*

In [2]:
instance = senseval.instances('hard.pos')[1]

# Let's check its fields
print('Target word: ', instance.word)
print('Position of the target word:', instance.position)
print('Senses: ', instance.senses)
print('Context of the target word, includes POS tags:', instance.context) 


Target word:  hard-a
Position of the target word: 10
Senses:  ('HARD1',)
Context of the target word, includes POS tags: [('clever', 'NNP'), ('white', 'NNP'), ('house', 'NNP'), ('``', '``'), ('spin', 'VB'), ('doctors', 'NNS'), ("''", "''"), ('are', 'VBP'), ('having', 'VBG'), ('a', 'DT'), ('hard', 'JJ'), ('time', 'NN'), ('helping', 'VBG'), ('president', 'NNP'), ('bush', 'NNP'), ('explain', 'VB'), ('away', 'RB'), ('the', 'DT'), ('economic', 'JJ'), ('bashing', 'NN'), ('that', 'IN'), ('low-and', 'JJ'), ('middle-income', 'JJ'), ('workers', 'NNS'), ('are', 'VBP'), ('taking', 'VBG'), ('these', 'DT'), ('days', 'NNS'), ('.', '.')]


Now let's load the data

In [3]:
import copy
from collections import defaultdict

instances = []
labels_classes = []  # List of (word, sense) tuples


# LOAD DATA AND SET OF LABELS (CLASSES)
for file in files_list:
    for i in senseval.instances(file):
        senses = i.senses

        # I had issues with the 'lazy' data loading approach, so I just copied the data directly into memory
        temp_instance = copy.deepcopy(i)  
        
        for sense in senses:  # Note that an instance may have more than one sense
            temp_instance.senses = sense
            instances.append(temp_instance)  # We create a separate entry for each sense
            labels_classes.append((i.word, sense))
label_sets = set(labels_classes)  # create our set of possible classes


We use ``label_sets`` as a set of the word senses in our data. We see that **hard** has 3 senses, **interest** has 6, **line** also has 6, and **serve** has 4.

In [4]:
label_sets

{('hard-a', 'HARD1'),
 ('hard-a', 'HARD2'),
 ('hard-a', 'HARD3'),
 ('interest-n', 'interest_1'),
 ('interest-n', 'interest_2'),
 ('interest-n', 'interest_3'),
 ('interest-n', 'interest_4'),
 ('interest-n', 'interest_5'),
 ('interest-n', 'interest_6'),
 ('line-n', 'cord'),
 ('line-n', 'division'),
 ('line-n', 'formation'),
 ('line-n', 'phone'),
 ('line-n', 'product'),
 ('line-n', 'text'),
 ('serve-v', 'SERVE10'),
 ('serve-v', 'SERVE12'),
 ('serve-v', 'SERVE2'),
 ('serve-v', 'SERVE6')}

# Feature Selection
We define two important variables, ``context``: the amount of words that are taken for each instance to analyze frequency (set to 2), and ``num_co_ocurrences``: the number of lemmas that co-occur with the target word, which is set to 12. Play around with these values to see how the results change.

We need to define a couple of functions.
``tokenize(text)`` function just tokenizes a text using the WordPunctTokenizer. We use it to get rid of punctuation and apostrophes. It may seem unnecessary, but we will re-use this function later when we'll use non-POS tagged data ;)


In [5]:
context = 2
num_co_ocurrences = 12

In [6]:
import string
from nltk import WordNetLemmatizer
from nltk.tokenize import WordPunctTokenizer
filters = ["``", "''"]  # we want to filter these from our data
lem = WordNetLemmatizer()
tokenizer = WordPunctTokenizer()
translator = str.maketrans('', '', string.punctuation)  # used to remove punctuation

def tokenize(text):
    tokenized_text = []
    for w in text:
        word = tokenizer.tokenize(w[0])[0].lower() # remove punctuation present in the text
        norm_word = word.translate(translator)                
        if len(norm_word) >= 1: 
            tokenized_text.append((norm_word, w[1]))  # returns tuple (word, POS)
    return tokenized_text

Next we define ``extract_context``. This function takes the surrounding tokens for a given position. So for a text *"I don't really like red apples"* for position 4, with a ``context=2`` the surrounding words are ``['really','like','apples']``.

The function returns the context to the left and right of the target word as a tuple.

In [7]:
def extract_context(target_position, text, context):
    # extract target word
    text_wo_target_left = [t for t in text[:target_position]]
    text_wo_target_right = [t for t in text[target_position+1:]]
    
    # tokenize
    text_wo_target_left = tokenize(text_wo_target_left)
    text_wo_target_right = tokenize(text_wo_target_right)
    
    # return extracted context
    return text_wo_target_left[-context:], text_wo_target_right[:context]        

Let's test this function with our data, we want to get the context of *'house'* with a window of 2:
Note that we removed punctutation

In [8]:
print(extract_context(2, instance.context, context))

([('clever', 'NNP'), ('white', 'NNP')], [('spin', 'VB'), ('doctors', 'NNS')])


Now we can define a function to create the features vector, ``extract_features``.

We create the vector using the target word, its context (surrounding words), the POS tags of the context, and the co-occurence vector (that we will show next)


In [9]:
def extract_features(targetword, left_ctxt, right_ctxt, co_occ_vector):
    context_text = ([i[0] for i in left_ctxt] + [i[0] for i in right_ctxt])
    context_pos = ([i[1] for i in left_ctxt] + [i[1] for i in right_ctxt])
    co_occurence_features = [1 if i in context_text else 0 for i in co_occ_vector]
    
    return {'word': targetword.split('.')[0],
            'co-occ': tuple(co_occurence_features),
            'context': tuple(context_text), 
            'pos_context': tuple(context_pos)
            }

    #  e.g. of a feature vector  
    # {'co-occ': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
    # 'context': ('toll', 'telephone', 'and', 'conduct')},
    # 'pos_context': ('JJ', 'NN', 'CC', 'NN')
    # 'word': 'lines'}


# Co-occurence vector
With these defined functions we now can compute a vector of the most frequent content words in the collection for each class, we use ``context`` to determine how many words from the left and right of the target word to take, and ``num_co_occurrences`` to determine the number of most frequent words to consider. To store this information we use a dictionary variable named ``co_occurences_per_classword`` using the lemma of the target-word as key and the vector as value. 

The co-occurrence vector is built using ``co_occurences_per_classword`` as reference, adding 1 if each term in the surrounding context is found on the reference, 0 otherwise. 

For example, the most common co-occurences for **bitter** are:
```
'hard-a': ['time','find','work', 'believe', 'get', 'make', 'imagine', 'look', 'say', 'way', 'would', 'take']
```

In [10]:
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.probability import FreqDist


num_co_ocurrences = 12
co_occurrences_per_classword = defaultdict(dict)

#  Train and Test set.
#  For testing classification on single classes we use separate sets for each class
test_set_per_class = defaultdict(dict)
train_set_per_class = defaultdict(dict)

# Get the most co-occurrences of lemmas to every class (excluding sense)
for classword in set([l[0] for l in label_sets]):
    co_occurrences_per_classword[classword] = []  # create entry in the dictionary
    lemmas_for_class = []  #  All possible lemmas for the class
    for instance in instances:
        #  get all text for given label
        if classword == instance.word:
            
            # get context on the left and right
            left_context, right_context = extract_context(instance.position, 
                                                          instance.context, context)
            
            # i[0] indicates we only take the first element of the tuple (word, pos)
            # We don't need the POS tags to build the co-occurrences vector.
            # We also filter stopwords.
            lemmas_for_class += [i[0] for i in left_context
                                 if i[0] not in stopwords.words('english')
                                 and len(i[0]) > 1]
            lemmas_for_class += [i[0] for i in right_context 
                                 if i[0] not in stopwords.words('english')
                                 and len(i[0]) > 1]

    # Compute most frequent occurrences
    dist = FreqDist(lemmas_for_class)  # Build the frequency disribution

    # Only take into account the most common occurrences (remember we initially set num_co_ocurrences = 12)
    cooccurrences = [l[0] for l in dist.most_common(num_co_ocurrences)]
    
    # Save this vector for this class
    co_occurrences_per_classword[classword] = cooccurrences[:num_co_ocurrences]

    train_set_per_class[classword] = []  # initialize
    test_set_per_class[classword] = []  # initialize


Let's check our results for word class **hard**

In [11]:
co_occurrences_per_classword['hard-a']

['time',
 'find',
 'work',
 'believe',
 'get',
 'make',
 'imagine',
 'look',
 'say',
 'way',
 'would',
 'take']

## Building our feature set (finally)
Now we can finally build our set of features. For each word class and its meanings (named here using the `label` variable) we extract the features for each instance and store them in the `feature_sets` list.
Once our feature set is completed, we store it into the set we are going to use to train or Naïve Bayes classifier. We'll use 80% of the data for training the classifier, and the remaining 20% to test it.

In [12]:
import random


for label in label_sets:
    feature_sets = [] 
    label_word = label[0]  # e.g. line-n
    label_sense = label[1]  # e.g. cord, division, formation, ...
    for instance in instances:
        if label_word == instance.word and label_sense in instance.senses:
            word = instance.context[instance.position][0]  # take only the word, not the POS
            left, right = (extract_context(instance.position, instance.context, context))
            feature_sets += [(extract_features(
                    word, left, right, 
                    co_occurrences_per_classword[label_word]), label)]
            #  e.g. of a feature:  
            # ({'co-occ': [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
            # 'context': ('toll', 'telephone', 'and', 'conduct')},
            # 'pos_context': ('JJ', 'NN', 'CC', 'NN')}),
            # ('line.n': 'phone'))  <---- this is the label (class) with the meaning
    
    # Shuffle the instances to distribute them on the train and test sets
    random.shuffle(feature_sets)
    
    #  We use 80% of the data for training, and 20% for testing
    sample = int(len(feature_sets)*0.2)
    
    #  For the purpose of testing classification on single classes, we use separate sets for each class
    #  For general classification, merge train_set_per_class and test_set_per_class
    
    training_set = feature_sets[sample+1:]
    train_set_per_class[label_word] += training_set
    test_set = feature_sets[:sample] 
    test_set_per_class[label_word] += test_set

We now have a set of features that we have distributed into `train_set_per_class` and `test_set_per_class`, let's check it out.

In [16]:
train_set_per_class['line-n'][1]

({'co-occ': (0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0),
  'context': ('a', 'telephone', 'open', 'to'),
  'pos_context': ('DT', 'NN', 'JJ', 'TO'),
  'word': 'line'},
 ('line-n', 'phone'))

This is an instance of **line**, where the word **telephone** also appears in the most common words for that class:

(Remember that the data is shuffled, when you run it again it's going to be different)

In [14]:
co_occurrences_per_classword['line-n']

['new',
 'telephone',
 'personal',
 'long',
 'computer',
 'car',
 'ibm',
 'fine',
 'draw',
 'computers',
 'one',
 'company']

# Training and Testing our classifier

Now that we have created our set of features and distributed them into training and test sets we can train our classifier using NLTK's `NaiveBayesClassifier`.

First let's train a classifier to disambiguate **hard** and see how accurate it is:


In [18]:
from nltk.classify import NaiveBayesClassifier


classifier = NaiveBayesClassifier.train(train_set_per_class['hard-a'])  #  Train for 'hard'

#  Now we can check the classifier accuracy using our remaining test data
print("Accuracy is: %s" % nltk.classify.accuracy(classifier, test_set_per_class['hard-a']))



Accuracy is: 0.792147806004619


We got an accuracy of 79%.

Now let's create a classifier using all the data of the corpus, that is using **hard, line, interest, serve**. To do it we merge the `train_set_per_class` and `test_set_per_class` that we built, so we can use all the data.

In [19]:
#  Merge train_set_per_class and test_set_per_class for classification on all classes.
#  We can use train_set_per_class[key] to test on a specific class (specific word/senses)
sub_train_set = []
for key in train_set_per_class.keys():
    sub_train_set += [i for i in train_set_per_class[key]]
sub_test_set = []
for key in test_set_per_class.keys():
    sub_test_set += [i for i in test_set_per_class[key]]
total = sub_train_set + sub_test_set

random.shuffle(total)
divide = int(len(total)*0.2)

train_set = total[divide:]
test_set = total[:divide]
classifier = NaiveBayesClassifier.train(train_set)
print("Accuracy of the classifier including all words is: %s" % nltk.classify.accuracy(classifier, test_set))


Accuracy of the classifier including all words is: 0.7142387372574811


___________________

## To wrap up
We were able to build a classifier that got us an accuracy of 71%.
In the next section we sill further evaluate the performance of the classifier, we'll perform **cross-validation** and also use a confusion matrix to compute Precision and Recall.